In [ ]:
%use dataframe(0.12.1), mysql(8.0.15)

import com.mysql.cj.jdbc.MysqlDataSource
import java.io.FileInputStream
import java.util.Properties

val properties = Properties()
val inputStream = FileInputStream("${System.getProperty("user.home")}/mysql.properties")
properties.load(inputStream)
inputStream.close()

val dataSource = MysqlDataSource()
dataSource.setUser(properties.getProperty("db.user"))
dataSource.setPassword(properties.getProperty("db.password"))
dataSource.setUrl(properties.getProperty("db.url"))

In [ ]:
fun runQuery(sql: String) = dataSource.getConnection().use { conn -> DataFrame.readSqlQuery(conn, sql) }
    
val rawCpct = runQuery(
    """select d.sampleId, biopsy.patientId, gender, birthYear, registrationDate, deathDate, primaryTumorLocation, primaryTumorType, hasSystemicPreTreatment, hasRadiotherapyPreTreatment, preTreatments, preTreatmentsType, preTreatmentsMechanism, d.treatmentGiven, d.radiotherapyGiven, treatmentStartDate, treatmentEndDate, treatment, consolidatedTreatmentType, concatenatedTreatmentType, consolidatedTreatmentMechanism, concatenatedTreatmentMechanism, d.firstResponse as firstResponseOrig, d.responseDate as firstResponseDateOrig, firstMatchedResponse.response as firstResponse, firstMatchedResponse.responseDate as firstResponseDate, firstMatchedPDResponse.response as firstResponsePD, firstMatchedPDResponse.responseDate as firstResponsePDDate, bestResponse
from sample s
inner join datarequest d on d.specimenId=s.specimenId
left join biopsy on biopsy.specimenId = s.specimenId
left join treatment on treatment.biopsyId = biopsy.id
left join (select * 
          from
          treatmentResponse as tr
          where measurementDone='Yes' and response not in ('ND','NE') and
          not exists ( 
            select * 
            from treatmentResponse as tr1
            where tr1.measurementDone='Yes' and tr1.response not in ('ND','NE') 
            and tr1.treatmentId = tr.treatmentId
		        and tr1.responseDate <= tr.responseDate
            and tr1.id != tr.id
        )
        and not(isnull(treatmentId))
) as firstMatchedResponse
on treatment.id = firstMatchedResponse.treatmentId
left join ( select *
            from
            treatmentResponse as tr
            where response = 'PD' and measurementDone= 'Yes' and
            not exists (
              select *
              from treatmentResponse as tr1
              where tr1.treatmentId = tr.treatmentId
              and tr1.responseDate <= tr.responseDate
              and tr1.id != tr.id
              and tr1.response = tr.response
          )
          and not(isnull(treatmentId))
) as firstMatchedPDResponse
on treatment.id = firstMatchedPDResponse.treatmentId
left join (select treatmentId, case when (group_concat(response) like '%CR%') then 'CR' when (group_concat(response) like '%PR%') then 'PR' when (group_concat(response) like '%SD%') then 'SD' when (group_concat(response) like '%PD%') then 'PD' else group_concat(response) end as bestResponse
            from
            treatmentResponse as tr
            where measurementDone= 'Yes' and response not in ('Non-CR/Non-PD', 'Clinical progression', 'ND', 'NE') and
            not exists (
              select *
              from treatmentResponse as tr1
              where tr1.treatmentId = tr.treatmentId
              and tr1.responseDate <= tr.responseDate
              and tr1.id != tr.id
              and tr1.response = tr.response
            )
            and not(isnull(treatmentId)) 
            group by 1
) as bestResponse
on treatment.id = bestResponse.treatmentId
where d.sampleId like 'CPCT%' order by registrationDate"""
)

In [ ]:
val cpctVariants = runQuery(
    """select dr.sampleId, krasG12Status, krasG13Status, krasNonG12G13Status, brafV600EStatus from datarequest dr
    left join (select distinct sampleId, 1 as krasG12Status from somaticVariant where reported and gene='KRAS' and canonicalHgvsProteinImpact like 'p.Gly12%') kg12 on dr.sampleId = kg12.sampleId and dr.sampleId like 'CPCT%'
    left join (select distinct sampleId, 1 as krasG13Status from somaticVariant where reported and gene='KRAS' and canonicalHgvsProteinImpact like 'p.Gly13%') kg13 on dr.sampleId = kg13.sampleId and dr.sampleId like 'CPCT%'
    left join (select distinct sampleId, 1 as krasNonG12G13Status from somaticVariant where reported and gene='KRAS' and canonicalHgvsProteinImpact not like 'p.Gly12%' and canonicalHgvsProteinImpact not like 'p.Gly13%') kngg on dr.sampleId = kngg.sampleId and dr.sampleId like 'CPCT%'
    left join (select distinct sampleId, 1 as brafV600EStatus from somaticVariant where gene='BRAF' and reported and canonicalHgvsProteinImpact='p.Val600Glu') braf on dr.sampleId = braf.sampleId and dr.sampleId like 'CPCT%'
    where dr.sampleId like 'CPCT%'
    """
    )

In [ ]:
val cpctDrivers = runQuery(
    """select a.sampleId,
if(krasStatus='positive', 0,1) as isKrasWildtype,
if(nrasStatus='positive', 0,1) as isNrasWildtype,
if(b2mStatus='positive', 0, 1) as isB2MWildtype,
if(erbb2AmpStatus='positive', 1,0) as hasErbb2Amp,
if(msStatus='MSI',1,0) as hasMsi,
tumorMutationalLoad,
if(driverCount is null, 0, driverCount) + if(fusionCount is null, 0, fusionCount) as totalDriverCount
from
(select sampleId from datarequest where sampleId like 'CPCT%') as a
left join (select sampleId, 'positive' as krasStatus from driverCatalog where driverLikelihood>0.8 and gene='KRAS') as b1 on a.sampleId=b1.sampleId
left join (select sampleId, 'positive' as nrasStatus from driverCatalog where driverLikelihood>0.8 and gene='NRAS' and driver='MUTATION') as c on a.sampleId=c.sampleId
left join (select sampleId, 'positive' as b2mStatus from driverCatalog where driverLikelihood>0.8 and gene='B2M') as d on a.sampleId=d.sampleId
left join (select sampleId, msStatus, tml as tumorMutationalLoad from purity) as f on a.sampleId=f.sampleId
left join (select sampleId, 'positive' as erbb2AmpStatus from driverCatalog where gene='ERBB2' and likelihoodMethod='AMP') as g on a.sampleId=g.sampleId
left join (select sampleId, count(*) as driverCount from driverCatalog where driverLikelihood>0.8 group by 1) as h on a.sampleId=h.sampleId
left join (select sampleId, count(*) as fusionCount from svFusion where reported group by 1) as i on a.sampleId=i.sampleId
"""
    )

In [ ]:
val cpctJoin = rawCpct.join(cpctDrivers) { sampleId match right.sampleId }
    .join(cpctVariants) { sampleId match right.sampleId }

In [ ]:
import java.time.temporal.ChronoUnit
import java.time.LocalDate
import java.util.Date

fun convertDate(date: Date) = LocalDate.parse(date.toString())

fun differenceInDays(firstDate: Date?, secondDate: Date?): Int? {
    return if (firstDate == null || secondDate == null) null else {
        ChronoUnit.MONTHS.between(convertDate(firstDate), convertDate(secondDate)).toInt()
    }
}

In [ ]:
val cpct = cpctJoin.add {
    "isFemale" from { (gender ?: "") == "female" }
    "ageAtTreatmentStart" from { treatmentStartDate?.let { LocalDate.parse(it.toString()).year - birthYear } }
    "hasImmunoPreTreatment" from { preTreatmentsType?.any { it.lowercase().contains("immunotherapy") } ?: false }
    "hasKrasG12Mut" from { krasG12Status != null }
    "hasKrasG13Mut" from { krasG13Status != null }
    "hasKrasNonG12G13Mut" from { krasNonG12G13Status != null }
    "isBRAFV600EWildtype" from { brafV600EStatus != null }
    "os" from { differenceInDays(treatmentStartDate, deathDate) } 
    "pfs" from { differenceInDays(treatmentStartDate, firstResponsePDDate) }
}
    .convert { hasSystemicPreTreatment }.with { it == "Yes" }
    .convert { hasMsi and isKrasWildtype and isNrasWildtype and isB2MWildtype and hasErbb2Amp }.with { it == 1L }
cpct.schema().print()

In [ ]:
@DataSchema
interface PatientRecord {
    val sampleId: String
    val patientId: Int
    val isFemale: Boolean
    val ageAtTreatmentStart: Int
    val treatment: String
    val hasSystemicPreTreatment: Boolean
    val hasImmunoPreTreatment: Boolean
    val primaryTumorLocation: String
    val isKrasWildtype: Boolean
    val isB2MWildtype: Boolean
    val hasMsi: Boolean
    val tumorMutationalLoad: Int
    val bestResponse: String?
    val pfs: Int?
    val os: Int?
}

In [ ]:
val pembroTreatments = setOf("Pembrolizumab", "Pembrolizumab/Pembrolizumab")
val pembroLocations = setOf("Urothelial tract", "Lung", "Skin", "Prostate", "Mesothelium")
val pembrolizumab: DataFrame<PatientRecord> = cpct.filter { treatment in pembroTreatments && primaryTumorLocation in pembroLocations }
    .select { sampleId and patientId and isFemale and ageAtTreatmentStart and treatment and
        hasSystemicPreTreatment and hasImmunoPreTreatment and primaryTumorLocation and isKrasWildtype and
        isB2MWildtype and hasMsi and tumorMutationalLoad and bestResponse and pfs and os }
    .cast<PatientRecord>()
pembrolizumab.schema().print()

In [ ]:
@file:DependsOn("nz.ac.waikato.cms.weka:weka-stable:3.8.6")

In [ ]:
pembrolizumab.filter { os != null && hasImmunoPreTreatment }.count()

In [ ]:
import weka.classifiers.functions.MultilayerPerceptron
import weka.core.Attribute
import weka.core.DenseInstance
import weka.core.Instance
import weka.core.Instances

fun setValue(instance: DenseInstance, attribute: Attribute, value: Any) {
    when (value) {
        is String -> instance.setValue(attribute, value)
        is Double -> instance.setValue(attribute, value)
        is Int -> instance.setValue(attribute, value.toDouble())
        is Boolean -> instance.setValue(attribute, if (value) 1.0 else 0.0)
        else -> throw IllegalArgumentException("Unsupported value type: ${value::class.simpleName}")
    }
}
    
fun createPatientDb(patients: DataFrame<PatientRecord>, classAttribute: String): Instances {
    val attributeMap = listOf(
        "isFemale",
        "ageAtTreatmentStart",
        "hasSystemicPreTreatment",
        "isKrasWildtype",
        "isB2MWildtype",
        "hasMsi",
        "tumorMutationalLoad",
        classAttribute
    ).associateWith { Attribute(it) }
    
    val attributes = attributeMap.values
    
    val patientDb = Instances("patients", ArrayList(attributeMap.values), patients.count())
    patientDb.setClassIndex(attributeMap[classAttribute]!!.index())
    
    patients.forEach { patient ->
        val patientInstance = DenseInstance(attributes.size)
        patientInstance.setDataset(patientDb)
        attributeMap.forEach { (column, attribute) -> setValue(patientInstance, attribute, patient[column]!!) }
        patientInstance.setClassValue((patient[classAttribute]!! as Int).toDouble())
        patientDb.add(patientInstance)
    }
    
    return patientDb
}

In [ ]:
val patientDb = createPatientDb(pembrolizumab.filter { os != null}, "os")

val predictor = MultilayerPerceptron()
// predictor.setHiddenLayers("4")
predictor.buildClassifier(patientDb)

In [ ]:
import weka.classifiers.evaluation.Evaluation
import java.util.Random

val evaluation = Evaluation(patientDb)
evaluation.crossValidateModel(predictor, patientDb, 10, Random())
println(evaluation.toSummaryString())

In [ ]:
import weka.classifiers.lazy.IBk

val classifier = IBk(1)
classifier.buildClassifier(patientDb)
val evaluation = Evaluation(patientDb)
evaluation.crossValidateModel(classifier, patientDb, 10, Random())
println(evaluation.toSummaryString())